![](../images/rivacon_frontmark_combined_header.png)

# Interest Rate Swap 

In [ ]:
import datetime as dt
import math

from rivapy.marketdata.curves import DiscountCurve
from rivapy.instruments import SwaptionSpecification
from rivapy.pricing import price
from pyvacon.finance.pricing import AnalyticSwaptionPricingData, PricingRequest
from pyvacon.finance.marketdata import LiborCurve, SwapCurve, SwaptionVolatilityCube
from pyvacon.finance.configuration import LiborIndex, SwapIndex
from pyvacon.finance.definition import SimpleHolidayCalendar

## Definition of an Interest Rate Swaption

A swaption is an instrument which gives the holder the right to enter into a swap at a pre-determined time. The most important attributes of a swaption are:
- Exercise style: If exercise into a swap is possible at only one fixed time, the swaption is called "European". If exercise is possible anytime, the swaption is called "American" (this is uncommon for swaptions). If exercise is possible at a discrete set of times, the swaption is called "Bermudan"
- Settlement: 
    - Physical settlement means that upon exercise the two counterparties enter into a swap
    - Cash settlement means that upon exercise the counterparty with the short position pays the present value of the (hypothetical) swap to the counterparty with the long position. In this case the swaption contract must specify how exactly the present value shall be calculated
- The attributes of the underlying swap (maturity, payment frequencies, reference index, fixed leg rate ...)


## Valuation of Interest Rate Swaptions

European swaptions are traded so frequently that quotes for them are readily available for a wide range of exercise dates, swap maturities and strikes. Prices are usually quoted as implied volatilities (either log-normal or normal). From these volatilities, prices are then obtained through the Black formula (for lognormal quotes) or Bachelier formula (for normal quotes). So pricing a European swaption boils down to interpolating the appropriate volatility from a volatility cube and then applying the Black/Bachelier formula.
Pricing Bermudan (or American) swaptions is more difficult; for that purpose, one needs a model for interest rates.

In [ ]:
# Setting up market data

ref_date = dt.datetime(2023, 12, 1)

# A (trivial) holiday calendar, with only weekends as holidays
cal = SimpleHolidayCalendar("cal")
cal.setWeekdayAsHoliday(6)  # Saturday
cal.setWeekdayAsHoliday(0)  # Sunday

# Discount curve
# (a flat curve with rate 1%)
object_id = "TEST_DC"
dsc_rate = 0.01
days_to_maturity = [1, 180, 365, 720, 3 * 365, 4 * 365, 10 * 365]
dates = [ref_date + dt.timedelta(days=d) for d in days_to_maturity]
df = [math.exp(-d / 365.0 * dsc_rate) for d in days_to_maturity]
dc = DiscountCurve(id=object_id, refdate=ref_date, dates=dates, df=df)

# Forward curve
# (a flat curve with rate 2%)
object_id = "TEST_6M"
fwd_rate = 0.02
days_to_maturity = [1, 180, 365, 720, 3 * 365, 4 * 365, 10 * 365]
dates = [ref_date + dt.timedelta(days=d) for d in days_to_maturity]
fwd_df = [math.exp(-d / 365.0 * fwd_rate) for d in days_to_maturity]
fwd_curve_6m = DiscountCurve(id=object_id, refdate=ref_date, dates=dates, df=fwd_df)

# The static data describing the underlying swap
euribor_6m_index = LiborIndex('eid', '6M', 'Act360', 'ModifiedFollowing', 2, cal)
euribor_6m_curve = LiborCurve("ec", ref_date, fwd_curve_6m._get_pyvacon_obj(), euribor_6m_index)
swap_index = SwapIndex('sid', euribor_6m_index,
                       '1Y', '30U360', 'ModifiedFollowing',
                       '6M', 'Act360', 'ModifiedFollowing',
                       cal)
swap_curve = SwapCurve("sc", ref_date, euribor_6m_curve, dc._get_pyvacon_obj(), swap_index)

# The vola cube (flat vola 20% lognormal)
vola_cube = SwaptionVolatilityCube.createFlatVolatilityCube("vola", ref_date, "LOGNORMAL", swap_curve, .2)


In [ ]:
# Setting up the swaption


ccy = 'EUR'
ex_date = dt.datetime(2025, 6, 1)
swap_mat = '5Y'
notional = 1.
strike = .02
settlement = 'PHYSICAL'
fix_dcc = '30U360'
fix_freq = '1Y'
fix_roll='ModifiedFollowing'
float_freq = '6M'
float_dcc = 'Act360'
float_roll='ModifiedFollowing'
float_rate_freq = '6M'
float_rate_dcc = 'Act360'
float_rate_roll='ModifiedFollowing'
swo = SwaptionSpecification.makeEuropeanSwaption(
    'id',
    'issuer',
    'COLLATERALIZED',
    ccy,
    ex_date,
    settlement,
    0.,  # forward premium
    0,  # spot days
    swap_mat,
    notional,
    strike,
    '', '', '', '',  # udl ids
    cal,
    cal,
    fix_freq,
    fix_dcc,
    fix_roll,
    'FLOAT',
    float_freq,
    float_dcc,
    float_roll,
    float_rate_freq,
    float_rate_dcc,
    float_rate_roll,
    0,
    0,
    True, # Payer swaption (=exercise into payer swap)
    True  # long position
)



In [ ]:
pricing_data = AnalyticSwaptionPricingData()
pricing_data.pricer = 'AnalyticSwaptionPricer'
pricing_data.pricingRequest = PricingRequest()
pricing_data.valDate = ref_date
pricing_data.spec = swo
pricing_data.discountCurve = dc._get_pyvacon_obj()
pricing_data.volCube = vola_cube


p = price(pricing_data)
p.getPrice()
